In [1]:
import os
from pathlib import Path
import openai
from dotenv import load_dotenv
from markdown import Markdown
import pandas as pd
import re
import tqdm
import asyncio
from openai import OpenAI
from agents import Agent, FileSearchTool, Runner, trace
from IPython.display import display, Markdown
load_dotenv()

True

## Pre-Process Excel Files

In [2]:
ALLOWED_EXTENSIONS = {
    ".c", ".cpp", ".cs", ".css", ".doc", ".docx", ".go", ".html", ".java",
    ".js", ".json", ".md", ".pdf", ".php", ".pptx", ".py", ".rb", ".sh",
    ".tex", ".ts", ".txt"
}

def get_all_file_paths(folder_path):
    """
    Recursively get all file paths in a folder and its subfolders.

    Args:
        folder_path (str or Path): Path to the root folder.

    Returns:
        list: A list of full file paths.
    """
    return [str(file) for file in Path(folder_path).rglob('*') if file.is_file()]

def get_allowed_file_paths(folder_path):
    """
    Recursively get file paths in a folder and its subfolders,
    filtered by allowed extensions.

    Args:
        folder_path (str or Path): Path to the root folder.

    Returns:
        list: A list of full file paths matching the allowed extensions.
    """
    return [
        str(file)
        for file in Path(folder_path).rglob('*')
        if file.is_file() and file.suffix.lower() in ALLOWED_EXTENSIONS
    ]

folder = "../Data/Knowledge_Base/Raw"
all_file_paths = get_all_file_paths(folder)
allowed_file_paths = get_allowed_file_paths(folder)
print("Number of files:", len(all_file_paths))
print("Number of Allowed files:", len(allowed_file_paths))

Number of files: 7
Number of Allowed files: 1


### Convert XLSX files to json

In [3]:
def excel_to_json_all_sheets(
    excel_path: str | Path,
    intermediate_dir: str | Path = "..//Data/Knowledge_Base/Intermediate",
    orient: str = "records",
    date_format: str = "iso",
    keep_index: bool = False
) -> list[Path]:
    """
    Convert every sheet in an Excel file to a JSON file in an intermediate folder.

    - Each sheet -> one JSON file.
    - JSON is an array of rows (orient='records' by default).
    - NaN -> null in JSON.
    - Dates -> ISO-8601 strings by default.

    Args:
        excel_path: Path to the .xlsx/.xls/.xlsm file.
        intermediate_dir: Folder to save JSON files (created if missing).
        orient: pandas DataFrame.to_json orient (default 'records').
        date_format: 'iso' (default) or 'epoch' for timestamps.
        keep_index: Whether to include the DataFrame index in the JSON.

    Returns:
        List of Paths to the saved JSON files.

    Raises:
        FileNotFoundError: If the Excel file does not exist.
        ValueError: If no sheets are found.
    """
    excel_path = Path(excel_path)
    if not excel_path.exists():
        raise FileNotFoundError(f"Excel file not found: {excel_path}")

    out_dir = Path(intermediate_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Read all sheets into a dict of {sheet_name: DataFrame}
    # sheet_name=None -> read all; engine auto-detected (requires openpyxl for .xlsx)
    sheets = pd.read_excel(excel_path, sheet_name=None)

    if not sheets:
        raise ValueError(f"No sheets found in {excel_path}")

    def _sanitize(name: str) -> str:
        # Keep it filesystem-friendly and stable
        name = name.strip()
        name = re.sub(r"\s+", "_", name)          # spaces -> underscores
        name = re.sub(r"[^A-Za-z0-9._-]", "", name)  # drop unsafe chars
        return name or "Sheet"

    saved_paths: list[Path] = []
    base = excel_path.stem

    for sheet_name, df in sheets.items():
        # Ensure consistent nulls in JSON; optional: drop all-empty columns/rows
        # df = df.dropna(how="all").loc[:, df.dropna(how="all").columns]
        safe_sheet = _sanitize(str(sheet_name))
        out_path = out_dir / f"{base}__{safe_sheet}.json"

        # Write JSON
        df.to_json(
            out_path,
            orient=orient,
            date_format=date_format,
            force_ascii=False,  # keep Unicode characters
            index=keep_index
        )
        saved_paths.append(out_path)

    return saved_paths

In [4]:
for file in tqdm.tqdm(all_file_paths):
    if file.lower().endswith((".xlsx", ".xlrd", ".xls", ".xlsm")):
        excel_to_json_all_sheets(file)

  0%|          | 0/7 [00:00<?, ?it/s]/Users/Francisco_Reveriano/Documents/Truist/Workforce_Model/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
 71%|███████▏  | 5/7 [00:03<00:01,  1.27it/s]/Users/Francisco_Reveriano/Documents/Truist/Workforce_Model/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/Francisco_Reveriano/Documents/Truist/Workforce_Model/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
100%|██████████| 7/7 [00:04<00:00,  1.71it/s]


## Create Assisstant

In [5]:
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"), base_url=os.getenv("OPENAI_API_BASE"))
# 1) Create a vector store to hold your PDFs
vs = client.vector_stores.create(name="Project PDFs")
# 2) Batch-upload many PDFs and wait until indexing is done
directory = "../Data/Knowledge_Base"
file_paths = get_allowed_file_paths(directory)
print(f"Uploading {len(file_paths)} Files...")
file_streams = [open(p, "rb") for p in file_paths]
try:
    batch = client.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vs.id,
        files=file_streams,  # any number of local PDFs
    )
    print(batch.status, batch.file_counts)  # e.g., "completed" and counts
finally:
    for f in file_streams:
        f.close()

['../Data/Knowledge_Base/Intermediate/Full_Hash_File (1)__FULL_HASH_FILE.json', '../Data/Knowledge_Base/Intermediate/Masked Extended_Worker IAM SDLC 06-30-25 (1)__IAM_L_Lang.json', '../Data/Knowledge_Base/Intermediate/Masked Extended_Worker IAM SDLC 06-30-25 (1)__SDLC_B_Stalnaker.json', '../Data/Knowledge_Base/Intermediate/ZBB_Data_Package (1)__Project_Info.json', '../Data/Knowledge_Base/Intermediate/AI_FILE1 (1)__AI_FILE.json', '../Data/Knowledge_Base/Intermediate/ZBB File (1)__ZBB_FILE.json', '../Data/Knowledge_Base/Intermediate/ZBB_Data_Package (1)__All_Team_Activities.json', '../Data/Knowledge_Base/Intermediate/ZBB_Data_Package (1)__Worker_Alignment_to_Team_.json', '../Data/Knowledge_Base/Intermediate/ZBB_Data_Package (1)__Team_Information.json', '../Data/Knowledge_Base/Intermediate/AI_FILE (1)__AI_FILE.json', '../Data/Knowledge_Base/Raw/June 2025 ST Monthly TDLC Metrics Report.pdf']
Uploading 11 Files...
completed FileCounts(cancelled=0, completed=10, failed=1, in_progress=0, tota

In [6]:
vs.id

'vs_68b76a37d41c8191885888e3b7e5cb1f'

## Setup Agent

In [7]:
import asyncio
from openai import OpenAI
from agents import Agent, FileSearchTool, Runner, trace
from IPython.display import display, Markdown

Process_Knowledge_Base = Agent(
        name="Process_Knowledge_Base",
        instructions="You are a helpful agent. You answer only based on the information in the vector store. Provide all citations with footnotes at the end of the answer.",
        model=os.getenv("LLM_MODEL"),
        tools=[
            FileSearchTool(
                max_num_results=20,
                vector_store_ids=[vs.id],
                include_search_results=True,
            )
        ],
    )

In [8]:
message = "Go through all the documents. Identify which individuals are involved in Identity Access & Management and provide a final headcount. And which of those roles can be augmented by Generative AI."
result = await Runner.run(Process_Knowledge_Base, message)
display(Markdown(result.final_output))

[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Your authentication token is not from a valid issuer.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_issuer"
  }
}


Here’s what I found across the uploaded material.

Individuals involved in Identity Access & Management (IAM) — by role/record
Core IAM organization (Identity & Access Management Division)
- Identity and Access Management Technical Director III — User Access Management and Engagement (Teammate) — 1 individual. 
- Identity and Access Management Technical Director III — Client Identity Access Management (Teammate) — 1 individual. 
- Identity and Access Management Technical Director II — Authentication & Authorization Management (Teammate) — 1 individual. 
- Identity and Access Management Technical Director I — Identity & Access Management Manager I (Teammate) — 1 individual. 
- Identity and Access Management Technical Director I — Role Based Access Control Lead Strategist, Access/Role Management (Teammate) — 1 individual.  
- Enterprise Tech – Cyber Security Analyst — User Access Lifecycle Support > Application Provisioning (Contingent workers; three distinct SOW records) — 3 individuals.   

Other individuals performing IAM activities (outside the IAM org)
- Worker Hash 4097E6E5… — Identity Access Management and “Centralize Access Administration.” 
- Worker Hash 19405D4D… — Identity Access Management and “Centralize Access Administration.” 
- Worker Hash 8061160C… — Identity Access Management and “Centralize Access Administration.” 
- Worker Hash C8DAD3DF… — Identity Access Management and “Centralize Access Administration.” 

Notes on potentially overlapping records
- A workforce-allocation record shows another IAM Technical Director III focused on governance, controls, standards, metrics, and compliance within the IAM Division. Because this dataset is separate and de-identified, and may overlap with the entries above, I did not add it to the core headcount to avoid double counting. 

Final headcount
- Core IAM organization (Identity & Access Management Division): 8 individuals (5 teammates + 3 contingent).
- Additional individuals performing IAM activities outside the IAM org: 4 individuals.
- Total involved in IAM across the documents (core + outside): 12 individuals.

Which of these roles can be augmented by Generative AI
- IAM Cloud Engineering: The team is explicitly driving “continued improvement for CICD and automation” and “leveraging AI to reduce engineering time.” This team’s scope includes Azure Active Directory (Entra), Role Management, Lifecycle items, CI/CD integration for IAM controls, and Privileged Access — all flagged as areas for optimization and automation. 
- AWS IAM Engineering & Operations: This team likewise highlights “continued improvement for CICD and automation” and the need for “additional upskilling for AI technology as related to IAM functions and automated workflow to limit human interaction,” indicating clear opportunities for GenAI augmentation across their IAM engineering and operations responsibilities. 
- Role Management/Access Modeling: The RBAC and role life-cycle work (e.g., “Role Design/Mining” and “Access Analysis and Reporting”) sits within IAM scope and is tied to the teams above that are already planning to leverage AI, making it a candidate for GenAI-assisted analysis/automation per the team charters.  

Footnotes
- Masked IAM teammate record — Identity & Access Management Technical Director I (User Access Lifecycle Support org details). 
- Masked IAM teammate record — Identity & Access Management Technical Director II (Authentication & Authorization Management). 
- Masked IAM teammate record — Identity & Access Management Technical Director III (User Access Management and Engagement). 
- Masked IAM teammate record — Identity & Access Management Technical Director III (Client Identity Access Management). 
- Masked IAM teammate record — Identity & Access Management Technical Director I (RBAC Lead Strategist; Access/Role Management). 
- Role Management activity definition (Access Model/Role Mining/SoD/Access Analysis). 
- Contingent worker records — IAM Provisioning/Deprovisioning & Access Mgmt (User Access Lifecycle Support > Application Provisioning).   
- Worker-alignments performing IAM activities (outside IAM org).  
- Workforce-allocation entry — IAM Technical Director III with IAM governance/metrics focus (possible overlap). 
- IAM Cloud Engineering team charter noting AI/automation for CICD and IAM controls. 
- AWS IAM Engineering & Operations team charter noting AI upskilling and automation to limit human interaction. 